# datasets

In [1]:
import h5py
import cupy as cp

In [2]:
#加载数据的function
def load_dataset():
    train_dataset = h5py.File('../datasets/train_signs.h5', "r")
    train_set_x_orig = cp.array(train_dataset["train_set_x"][:])  # your train set features
    train_set_y_orig = cp.array(train_dataset["train_set_y"][:])  # your train set labels

    test_dataset = h5py.File('../datasets/test_signs.h5', "r")
    test_set_x_orig = cp.array(test_dataset["test_set_x"][:])  # your test set features
    test_set_y_orig = cp.array(test_dataset["test_set_y"][:])  # your test set labels

    classes = cp.array(test_dataset["list_classes"][:])  # the list of classes

    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))

    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes


In [3]:
#加载数据
train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes=load_dataset()

trainX=train_set_x_orig/255  #shape(1080,64,64,3)样本数，高，宽，通道数
trainX=trainX.transpose(0,3,1,2)

testX=test_set_x_orig/255
testX=testX.transpose(0,3,1,2)

trainy=train_set_y_orig
testy=test_set_y_orig
#转为one-hot编码

n_class = cp.max(testy).tolist() + 1
trainy=cp.eye(n_class)[trainy].reshape(-1,6)
testy=cp.eye(n_class)[testy].reshape(-1,6)


In [4]:
print('trainX shape:',trainX.shape)
print('trainy shape:',trainy.shape)
print('testX shape:',testX.shape)
print('testy shape:',testy.shape)

trainX shape: (1080, 3, 64, 64)
trainy shape: (1080, 6)
testX shape: (120, 3, 64, 64)
testy shape: (120, 6)


# 1.shinnosuke functional model

In [7]:
from shinnosuke.layers.Convolution import Conv2D,MaxPooling2D,MeanPooling2D
from shinnosuke.layers.Activation import Activation
from shinnosuke.layers.Normalization import BatchNormalization
from shinnosuke.layers.Dropout import Dropout
from shinnosuke.layers.FC import Flatten,Dense
from shinnosuke.layers.Base import Input
from shinnosuke.models import Model

In [6]:
X_input=Input(shape=(None,3,64,64))
X=Conv2D(8,(5,5),padding='VALID',initializer='normal',activation='relu')(X_input)
X=BatchNormalization(axis=1)(X)
X=MaxPooling2D((4,4))(X)
X=Conv2D(16,(3,3),padding='SAME',initializer='normal',activation='relu')(X)
X=MaxPooling2D((4,4))(X)
X=Flatten()(X)
X=Dense(6,initializer='normal',activation='softmax')(X)
model=Model(inputs=X_input,outputs=X)
model.compile(optimizer='sgd',loss='sparse_categorical_cross_entropy')
model.fit(trainX,trainy,batch_size=256,epochs=100,validation_ratio=0.)

 Epoch[1/100]
1280/1280 [==============================>] -3s -573ms/batch -batch_loss: 1.7614 -batch_acc: 0.2500 
 Epoch[2/100]
1280/1280 [==============================>] -2s -322ms/batch -batch_loss: 1.7544 -batch_acc: 0.2321 
 Epoch[3/100]
1280/1280 [==============================>] -1s -293ms/batch -batch_loss: 1.7901 -batch_acc: 0.2679 
 Epoch[4/100]
1280/1280 [==============================>] -1s -281ms/batch -batch_loss: 1.4067 -batch_acc: 0.4643 
 Epoch[5/100]
1280/1280 [==============================>] -1s -273ms/batch -batch_loss: 1.3655 -batch_acc: 0.5000 
 Epoch[6/100]
1280/1280 [==============================>] -2s -326ms/batch -batch_loss: 1.4472 -batch_acc: 0.3929 
 Epoch[7/100]
1280/1280 [==============================>] -2s -335ms/batch -batch_loss: 1.4015 -batch_acc: 0.3036 
 Epoch[8/100]
1280/1280 [==============================>] -2s -327ms/batch -batch_loss: 1.2613 -batch_acc: 0.5357 
 Epoch[9/100]
1280/1280 [==============================>] -2s -322ms/batch -batc

In [9]:
acc,loss=model.evaluate(testX,testy)
print('test acc: %f,test loss: %f'%(acc,loss))

test acc: 0.808333,test loss: 0.560832


# shinnosuke sequential model 

In [8]:
from shinnosuke.models import Sequential

In [9]:
m=Sequential()
m.add(Conv2D(8,(5,5),input_shape=(None,3,64,64),padding='VALID',initializer='normal'))
m.add(Activation('relu'))
m.add(BatchNormalization(axis=1))
m.add(MaxPooling2D((4,4)))
m.add(Conv2D(16,(3,3),padding='VALID',initializer='normal'))
m.add(Activation('relu'))
m.add(MaxPooling2D((4,4)))
m.add(Flatten())
m.add(Dense(6,initializer='normal',activation='softmax'))
m.compile(optimizer='sgd',loss='sparse_categorical_cross_entropy')
m.fit(trainX,trainy,batch_size=256,epochs=100,validation_ratio=0.)

 Epoch[1/100]
1280/1280 [==============================>] -3s -580ms/batch -batch_loss: 1.8922 -batch_acc: 0.1607 
 Epoch[2/100]
1280/1280 [==============================>] -2s -315ms/batch -batch_loss: 1.6832 -batch_acc: 0.2500 
 Epoch[3/100]
1280/1280 [==============================>] -1s -291ms/batch -batch_loss: 1.5676 -batch_acc: 0.3214 
 Epoch[4/100]
1280/1280 [==============================>] -1s -278ms/batch -batch_loss: 1.3879 -batch_acc: 0.4286 
 Epoch[5/100]
1280/1280 [==============================>] -1s -274ms/batch -batch_loss: 1.2279 -batch_acc: 0.5179 
 Epoch[6/100]
1280/1280 [==============================>] -1s -275ms/batch -batch_loss: 1.1043 -batch_acc: 0.5357 
 Epoch[7/100]
1280/1280 [==============================>] -1s -276ms/batch -batch_loss: 1.4078 -batch_acc: 0.3929 
 Epoch[8/100]
1280/1280 [==============================>] -1s -275ms/batch -batch_loss: 1.3480 -batch_acc: 0.3214 
 Epoch[9/100]
1280/1280 [==============================>] -1s -275ms/batch -batc

1280/1280 [==============================>] -1s -292ms/batch -batch_loss: 0.7445 -batch_acc: 0.7321 
 Epoch[68/100]
1280/1280 [==============================>] -2s -336ms/batch -batch_loss: 0.6848 -batch_acc: 0.7500 
 Epoch[69/100]
1280/1280 [==============================>] -2s -333ms/batch -batch_loss: 0.8459 -batch_acc: 0.6964 
 Epoch[70/100]
1280/1280 [==============================>] -2s -315ms/batch -batch_loss: 0.6364 -batch_acc: 0.7679 
 Epoch[71/100]
1280/1280 [==============================>] -1s -290ms/batch -batch_loss: 0.6133 -batch_acc: 0.7321 
 Epoch[72/100]
1280/1280 [==============================>] -1s -279ms/batch -batch_loss: 0.9783 -batch_acc: 0.5536 
 Epoch[73/100]
1280/1280 [==============================>] -1s -274ms/batch -batch_loss: 0.6489 -batch_acc: 0.7679 
 Epoch[74/100]
1280/1280 [==============================>] -1s -275ms/batch -batch_loss: 0.7306 -batch_acc: 0.7321 
 Epoch[75/100]
1280/1280 [==============================>] -1s -275ms/batch -batch_loss

In [10]:
acc,loss=m.evaluate(testX,testy)
print('test acc: %f,test loss: %f'%(acc,loss))

test acc: 0.700000,test loss: 0.706504


# keras-gpu functional model

In [14]:
trainX=cp.asnumpy(trainX)
trainy=cp.asnumpy(trainy)
testX=cp.asnumpy(testX)
testy=cp.asnumpy(testy)

In [18]:
import keras
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Flatten,Input,Conv2D, MaxPooling2D,BatchNormalization,Activation

In [22]:
X_input=Input(shape=(3,64,64))
X=Conv2D(8,(5,5),padding='VALID',kernel_initializer='normal',activation='relu',data_format='channels_first')(X_input)
X=BatchNormalization(axis=1)(X)
X=MaxPooling2D((4,4))(X)
X=Conv2D(16,(3,3),padding='SAME',kernel_initializer='normal',activation='relu',data_format='channels_first')(X)
X=MaxPooling2D((4,4))(X)
X=Flatten()(X)
X=Dense(6,kernel_initializer='normal',activation='softmax')(X)
model=Model(inputs=X_input,outputs=X)
model.compile(optimizer='sgd',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(trainX,trainy,batch_size=256,epochs=100)

Epoch 1/100
1080/1080 [==============================] - 1s 1ms/step - loss: 1.8608 - acc: 0.1889
Epoch 2/100
1080/1080 [==============================] - 1s 583us/step - loss: 1.7986 - acc: 0.1917
Epoch 3/100
1080/1080 [==============================] - 1s 567us/step - loss: 1.7872 - acc: 0.2074
Epoch 4/100
1080/1080 [==============================] - 1s 554us/step - loss: 1.7787 - acc: 0.2111
Epoch 5/100
1080/1080 [==============================] - 1s 550us/step - loss: 1.7758 - acc: 0.2250
Epoch 6/100
1080/1080 [==============================] - 1s 537us/step - loss: 1.7685 - acc: 0.2352
Epoch 7/100
1080/1080 [==============================] - 1s 536us/step - loss: 1.7628 - acc: 0.2306
Epoch 8/100
1080/1080 [==============================] - 1s 515us/step - loss: 1.7573 - acc: 0.2435
Epoch 9/100
1080/1080 [==============================] - 1s 528us/step - loss: 1.7511 - acc: 0.2491
Epoch 10/100
1080/1080 [==============================] - 1s 516us/step - loss: 1.7461 - acc: 0.2509
E

1080/1080 [==============================] - 1s 498us/step - loss: 0.9213 - acc: 0.6537
Epoch 83/100
1080/1080 [==============================] - 1s 498us/step - loss: 0.8685 - acc: 0.6815
Epoch 84/100
1080/1080 [==============================] - 1s 496us/step - loss: 0.8933 - acc: 0.6556
Epoch 85/100
1080/1080 [==============================] - 1s 498us/step - loss: 0.8671 - acc: 0.6806
Epoch 86/100
1080/1080 [==============================] - 1s 497us/step - loss: 0.8792 - acc: 0.6583
Epoch 87/100
1080/1080 [==============================] - 1s 495us/step - loss: 0.8671 - acc: 0.6880
Epoch 88/100
1080/1080 [==============================] - 1s 491us/step - loss: 0.9144 - acc: 0.6685
Epoch 89/100
1080/1080 [==============================] - 1s 497us/step - loss: 0.9552 - acc: 0.6157
Epoch 90/100
1080/1080 [==============================] - 1s 500us/step - loss: 0.9149 - acc: 0.6491
Epoch 91/100
1080/1080 [==============================] - 1s 498us/step - loss: 0.8061 - acc: 0.7019
Epo

In [23]:
loss,acc=model.evaluate(testX,testy)
print('test acc: %f,test loss: %f'%(acc,loss))

120/120 [==============================] - 0s 3ms/step
test acc: 0.550000,test loss: 1.100569


# keras-gpu sequential model

In [26]:
m=Sequential()
m.add(Conv2D(8,(5,5),input_shape=(3,64,64),padding='VALID',kernel_initializer='normal',data_format='channels_first'))
m.add(Activation('relu'))
m.add(BatchNormalization(axis=1))
m.add(MaxPooling2D((4,4)))
m.add(Conv2D(16,(3,3),padding='VALID',kernel_initializer='normal',data_format='channels_first'))
m.add(Activation('relu'))
m.add(MaxPooling2D((4,4)))
m.add(Flatten())
m.add(Dense(6,kernel_initializer='normal',activation='softmax'))
m.compile(optimizer='sgd',loss='categorical_crossentropy',metrics=['accuracy'])
m.fit(trainX,trainy,batch_size=256,epochs=100)

Epoch 1/100
1080/1080 [==============================] - 2s 2ms/step - loss: 1.7960 - acc: 0.1713
Epoch 2/100
1080/1080 [==============================] - 1s 541us/step - loss: 1.7808 - acc: 0.1889
Epoch 3/100
1080/1080 [==============================] - 1s 526us/step - loss: 1.7734 - acc: 0.2065
Epoch 4/100
1080/1080 [==============================] - 1s 519us/step - loss: 1.7631 - acc: 0.2250
Epoch 5/100
1080/1080 [==============================] - 1s 516us/step - loss: 1.7538 - acc: 0.2269
Epoch 6/100
1080/1080 [==============================] - 1s 513us/step - loss: 1.7477 - acc: 0.2500
Epoch 7/100
1080/1080 [==============================] - 1s 512us/step - loss: 1.7276 - acc: 0.2731
Epoch 8/100
1080/1080 [==============================] - 1s 508us/step - loss: 1.7134 - acc: 0.2898
Epoch 9/100
1080/1080 [==============================] - 1s 510us/step - loss: 1.6983 - acc: 0.2991
Epoch 10/100
1080/1080 [==============================] - 1s 508us/step - loss: 1.6921 - acc: 0.3046
E

1080/1080 [==============================] - 1s 493us/step - loss: 0.6552 - acc: 0.7611
Epoch 83/100
1080/1080 [==============================] - 1s 491us/step - loss: 0.5664 - acc: 0.8130
Epoch 84/100
1080/1080 [==============================] - 1s 492us/step - loss: 0.5582 - acc: 0.8176
Epoch 85/100
1080/1080 [==============================] - 1s 491us/step - loss: 0.5759 - acc: 0.7972
Epoch 86/100
1080/1080 [==============================] - 1s 489us/step - loss: 0.5070 - acc: 0.8287
Epoch 87/100
1080/1080 [==============================] - 1s 492us/step - loss: 0.5065 - acc: 0.8380
Epoch 88/100
1080/1080 [==============================] - 1s 498us/step - loss: 0.5145 - acc: 0.8287
Epoch 89/100
1080/1080 [==============================] - 1s 491us/step - loss: 0.5689 - acc: 0.8056
Epoch 90/100
1080/1080 [==============================] - 1s 494us/step - loss: 0.5914 - acc: 0.7880
Epoch 91/100
1080/1080 [==============================] - 1s 492us/step - loss: 0.5092 - acc: 0.8343
Epo

In [28]:
loss,acc=m.evaluate(testX,testy)
print('test acc: %f,test loss: %f'%(acc,loss))

120/120 [==============================] - 0s 625us/step
test acc: 0.391667,test loss: 2.013439
